# Data Exploration

In this notebook describe your data exploration steps.

## Install dependencies

In [ ]:
%pip install pandas
%pip install plotly
%pip install nbformat

## Load data

In [ ]:
# import the modules
import sqlite3
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# connect to the database
con = sqlite3.connect("../data/data.sqlite")
 
# The following manipulations are done in SQL
# All columns from the table pedestrians are beeing selected and some renamed, but from the column 'time of measurement' only the first 10 characters are selected
# Additionally a column with row numbers is added to the table

# As the temperature and rain data are stored in two different tables, the data is combined in two subqueries
# The average of the temperature and rain data is calculated and a column with row numbers is added to the tables

# The three subqueries are joined together and the columns 'time', 'weekday', 'pedestrians', 'rain' and 'temperature' are selected

df = pd.read_sql_query('''

WITH PedestrianData AS (
    SELECT
        SUBSTR(p.[time of measurement], 1, 10) AS day,
        p.weekday AS weekday,
        p.[pedestrians count] AS pedestrians,
        ROW_NUMBER() OVER (ORDER BY p.[time of measurement]) AS row_num
    FROM pedestrians p
),
RainData AS (
    SELECT
        (r1.[Niederschlag (6 bis 6 UTC)] + r2.[Niederschlag (6 bis 6 UTC)]) / 2 AS rain,
        ROW_NUMBER() OVER (ORDER BY r1.[Niederschlag (6 bis 6 UTC)]) AS row_num
    FROM rainmoe r1, rainnue r2
),
TemperatureData AS (
    SELECT
        (t1.[Mittelwert] + t2.[Mittelwert]) / 2 AS temperature,
        ROW_NUMBER() OVER (ORDER BY t1.[Mittelwert]) AS row_num
    FROM tempmoe t1, tempnue t2
)

SELECT
    pd.day,
    pd.weekday,
    pd.pedestrians,
    rd.rain,
    td.temperature
FROM
    PedestrianData pd
JOIN
    RainData rd ON pd.row_num = rd.row_num
JOIN
    TemperatureData td ON pd.row_num = td.row_num;

''', con)

### Look at the first rows

In [ ]:
df.head(20)

### Data exploration
Print some basic information about the data. Your data exploration would continue here.

In [ ]:
df.info()

In [ ]:
# print min and max values of the columns

print("Min Pedestrians: " + str(df['pedestrians'].min()))
print("Max Pedestrians: " + str(df['pedestrians'].max()))
print("Min Rain: " + str(df['rain'].min()))
print("Max Rain: " + str(df['rain'].max()))
print("Min Temperature: " + str(df['temperature'].min()))
print("Max Temperature: " + str(df['temperature'].max()))

### Print some figures

In [ ]:
fig = px.scatter(
    df,
    x="temperature",
    y="rain",
    size="pedestrians",
    color="pedestrians",
    hover_data=["day"],
    labels={'day':'Date', 'pedestrians': 'Number of Pedestrians', 'temperature': 'Temperature in °C', 'rain': 'Rain in mm'},
    title="Rainfall and temperature in relation to the number of pedestrians."
)
fig.show()

fig = px.bar(
    df,
    x="day",
    y="pedestrians",
    color="rain",
    barmode="group",
    labels={'day':'Date', 'pedestrians': 'Number of Pedestrians', 'rain': 'Rain in mm'},
    title="Number of pedestrians per day colored with rainfall in mm."
)
fig.show()

fig = px.bar(
    df,
    x="day",
    y="pedestrians",
    color="temperature",
    barmode="group",
    labels={'day':'Date', 'pedestrians': 'Number of Pedestrians', 'temperature': 'Temperature in °C'},
    title="Number of pedestrians per day colored with temperature in °C."
)
fig.show()